<a href="https://colab.research.google.com/github/sudhirshahu51/LangGraph/blob/main/1_Simple_ReAct_Agent_from_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


1. Installations





In [3]:
!pip install openai python-dotenv httpx langchain-groq --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.7/412.7 kB 14.7 MB/s eta 0:00:00


2. Importing Libraries

In [4]:
import openai
import re
import httpx
import os
from openai import OpenAI

In [5]:
from google.colab import userdata
groq_key = userdata.get('GROQ_API')

#from langchain_groq import ChatGroq
#llm = ChatGroq(model="llama3-70b-8192", api_key=groq_key)

In [15]:
client = OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=groq_key
)

In [12]:
chat_completion = client.chat.completions.create(
    model="llama-3.3-70b-versatile",
    messages=[{"role": "user", "content": "Hello world"}]
)

In [13]:
chat_completion.choices[0].message.content

"Hello! It's nice to meet you. Is there something I can help you with or would you like to chat?"

In [16]:
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(
                        model="llama-3.3-70b-versatile",
                        temperature=0,
                        messages=self.messages)
        return completion.choices[0].message.content


In [18]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:

Answer: A bulldog weights 51 lbs
""".strip()

In [19]:
def calculate(what):
    return eval(what)

def average_dog_weight(name):
    if name in "Scottish Terrier":
        return("Scottish Terriers average 20 lbs")
    elif name in "Border Collie":
        return("a Border Collies average weight is 37 lbs")
    elif name in "Toy Poodle":
        return("a toy poodles average weight is 7 lbs")
    else:
        return("An average dog weights 50 lbs")

known_actions = {
    "calculate": calculate,
    "average_dog_weight": average_dog_weight
}

In [20]:
abot = Agent(prompt)

In [21]:
result = abot("How much does a toy poodle weigh?")
print(result)

Thought: I should look up the average weight of a toy poodle using the average_dog_weight action to find the correct weight.
Action: average_dog_weight: Toy Poodle
PAUSE


In [22]:
result = average_dog_weight("Toy Poodle")

In [23]:
result

'a toy poodles average weight is 7 lbs'

In [24]:
next_prompt = "Observation: {}".format(result)

In [25]:
abot(next_prompt)

'Answer: A toy poodle weighs 7 lbs.'

In [26]:
abot.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\naverage_dog_weight:\ne.g. average_dog_weight: Collie\nreturns average weight of a dog when given the breed\n\nExample session:\n\nQuestion: How much does a Bulldog weigh?\nThought: I should look the dogs weight using average_dog_weight\nAction: average_dog_weight: Bulldog\nPAUSE\n\nYou will be called again with this:\n\nObservation: A Bulldog weights 51 lbs\n\nYou then output:\n\nAnswer: A bulldog weights 51 lbs'},
 {'role': 'user', 'content': 'How much does a 

In [27]:
abot = Agent(prompt)

In [28]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
abot(question)

'Thought: To find the combined weight of the two dogs, I need to find the average weight of a Border Collie and a Scottish Terrier, then add the two weights together. I can use the average_dog_weight action to find the weight of each breed.\n\nAction: average_dog_weight: Border Collie\nPAUSE'

In [29]:
next_prompt = "Observation: {}".format(average_dog_weight("Border Collie"))
print(next_prompt)

Observation: a Border Collies average weight is 37 lbs


In [30]:
abot(next_prompt)

'Thought: Now that I have the average weight of a Border Collie, I need to find the average weight of a Scottish Terrier. I can use the average_dog_weight action again to find this weight. Once I have both weights, I can add them together to find the combined weight.\n\nAction: average_dog_weight: Scottish Terrier\nPAUSE'

In [31]:
next_prompt = "Observation: {}".format(eval("37 + 20"))
print(next_prompt)

Observation: 57


In [32]:
abot(next_prompt)

"Thought: I now have the average weights of both breeds. The Border Collie weighs 37 lbs and the Scottish Terrier weighs 57 lbs (I assume the unit is pounds, as it was for the Border Collie, although it wasn't explicitly stated in the observation). To find the combined weight, I can add these two weights together.\n\nAction: calculate: 37 + 57\nPAUSE"

Add loop

In [33]:
action_re = re.compile('^Action: (\w+): (.*)$')   # python regular expression to selection action

In [34]:
def query(question, max_turns=5):
    i = 0
    bot = Agent(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [
            action_re.match(a)
            for a in result.split('\n')
            if action_re.match(a)
        ]
        if actions:
            # There is an action to run
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = "Observation: {}".format(observation)
        else:
            return

In [35]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
query(question)

Thought: To find the combined weight of the two dogs, I need to find the average weight of a Border Collie and a Scottish Terrier, then add the two weights together. I can use the average_dog_weight action to find the weight of each breed.

Action: average_dog_weight: Border Collie
PAUSE
 -- running average_dog_weight Border Collie
Observation: a Border Collies average weight is 37 lbs
Thought: Now that I have the average weight of a Border Collie, I need to find the average weight of a Scottish Terrier. I can use the average_dog_weight action again to find this weight.

Action: average_dog_weight: Scottish Terrier
PAUSE
 -- running average_dog_weight Scottish Terrier
Observation: Scottish Terriers average 20 lbs
Thought: Now that I have the average weights of both breeds, I can add them together to find the combined weight of the two dogs.

Action: calculate: 37 + 20
PAUSE
 -- running calculate 37 + 20
Observation: 57
Thought: The calculation has given me the combined weight of the tw